In [6]:
class Stack:
    def __init__(self):
        self.top = None
        self.size = 0

    def __len__(self):
        return self.size

    def __iter__(self):
        return self.Iter(self.top)

    class Iter:
        def __init__(self, top):
            self.current = top

        def __iter__(self):
            return self

        def __next__(self):
            if self.current == None:
                raise StopIteration
            result = self.current.value
            self.current = self.current.next
            return result

    class NodeStack:
        def __init__(self, value):
            self.value = value
            self.next = None

    def __repr__(self):
        represent = ''
        current = self.top
        while current:
            represent += f'{current.value} --> '
            current = current.next
        return represent

    def push(self, value):
        new_node = self.NodeStack(value)
        if not self.size:
            self.top = new_node
        else:
            new_node.next = self.top
            self.top = new_node
        self.size += 1

    def pop(self):
        if not self.size:
            return
        popped_node = self.top
        self.top = self.top.next
        self.size -= 1
        return popped_node.value

    def includes(self, value):
        current = self.top
        while current:
            if current.value == value:
                return True
            current = current.next
        return False

    def __contains__(self, value):
        return self.includes(value)


# A --> C
# |     |
# \/    \/
# B     E
# |
# \/
# D --> F


graph = {
    'a': ['b', 'c'],
    'b': ['d'],
    'c': ['e'],
    'd': ['f'],
    'e': [],
    'f': []
}


def depth_first(graph, source):
    stack = Stack()
    stack.push(source)
    while stack.size:
        popped = stack.pop()
        print(popped)
        for neighbor in graph[popped][::-1]:
            stack.push(neighbor)


def recursive_depth_first(graph, source):
    print(source)
    for neighbor in graph[source]:
        recursive_depth_first(graph, neighbor)

    
graph = {
    'f': ['g', 'i'],
    'g': ['h'],
    'h': [],
    'i': ['g', 'k'],
    'j': ['i'],
    'k': []
}


def has_path(graph, src, dst):
    stack = Stack()
    stack.push(src)
    while stack.size:
        popped = stack.pop()
        if popped == dst:
            return True
        for neighbor in graph[popped][::-1]:
            stack.push(neighbor)
    return False


def has_path_bk(graph, src, dst):
    stack = Stack()
    stack_bk = Stack()
    stack.push(src)
    while stack.size:
        popped = stack.pop()
        if not stack_bk.includes(popped):
            if popped == dst:
                return True
            for neighbor in graph[popped][::-1]:
                stack.push(neighbor)
            stack_bk.push(popped)
    return False


def has_path_recursive(graph, src, dst):
    if src == dst:
        return True
    for neighbor in graph[src]:
        if has_path_recursive(graph, neighbor, dst):
            return True
    return False


def hpr(graph, src, dst):
    if src == dst:
        return True
    return any(hpr(graph, neighbor, dst) for neighbor in graph[src])


edges = [
  ['i', 'j'],
  ['k', 'i'],
  ['m', 'k'],
  ['k', 'l'],
  ['o', 'n'],
]


def build_graph1(edges):
    graph = dict()
    for edge in edges:
        for node_value in edge:
            new_values = [value for value in edge if value != node_value]
            try:
                graph[node_value] = [*graph[node_value], *new_values]
            except KeyError:
                graph[node_value] = new_values
    return graph


def build_graph(edges):
    graph = dict()
    for edge in edges:
        [first, second] = edge
        if first not in graph:
            graph[first] = []
        if second not in graph:
            graph[second] = []
        graph[first].append(second)
        graph[second].append(first)
    return graph


def recursive_build_graph(edges):
    graph = dict()
    def inner(index=0):
        try:
            [first, second] = edges[index]
            if first not in graph:
                graph[first] = []
            if second not in graph:
                graph[second] = []
            graph[first].append(second)
            graph[second].append(first)
            inner(index + 1)
        except IndexError:
            return
    inner()
    return graph


# {
#   i: [j, k],
#   j: [i],
#   k: [i, m, l],
#   m: [k],
#   l: [k],
#   o: [n],
#   n: [o]
# }


graph = build_graph(edges)


def undirected_hp(graph, src, dst):
    stack = Stack()
    stack_bk = Stack()
    stack.push(src)
    while stack.size:
        popped = stack.pop()
        if not stack_bk.includes(popped):
            if popped == dst:
                return True
            for neighbor in graph[popped]:
                stack.push(neighbor)
            stack_bk.push(popped)
    return False


def undi_hp_recur(graph, src, dst):
    stack_bk = Stack()
    def inner(src):
        if stack_bk.includes(src):
            return False
        stack_bk.push(src)
        if src == dst:
            return True
        for neighbor in graph[src]:
            if inner(neighbor):
                return True
        return False
    return inner(src)


def uhr(graph, src, dst):
    stack_bk = Stack()
    def inner(src):
        if stack_bk.includes(src):
            return False
        stack_bk.push(src)
        if src == dst:
            return True
        return any(inner(neighbor) for neighbor in graph[src])
    return inner(src)


graph = {
    3: [],
    4: [6],
    6: [4, 5, 7, 8],
    8: [6],
    7: [6],
    5: [6],
    1: [2],
    2: [1],
}


def recur_traver(graph, src):
    stack_bk = Stack()
    def inner(src):
        if src not in stack_bk:
            return
        stack_bk.push(src)
        for neighbor in graph[src]:
            inner(neighbor)
    inner(src)
    return stack_bk


def connected_components_count(graph):
    count = 0
    stack_bk = Stack()
    for key in graph:
        if key not in stack_bk:
            stack_bk = [*stack_bk, *recur_traver(graph, key)]
            count += 1
    return count


def recursive_connected(graph):
    keys = list(graph.keys())
    def inner(stack_bk, count, index=0):
        try:
            key = keys[index]
            if key not in stack_bk:
                stack_bk = [*stack_bk, *recur_traver(graph, key)]
                count += 1
            return inner(stack_bk, count, index + 1)
        except IndexError:
            return count
    return inner(Stack(), 0)


def count_components(graph):
    count = 0
    stack_bk = Stack()
    for key in graph:
        if explore(graph, key, stack_bk):
            count += 1
    return count


def explore(graph, src, stack_bk):
    if src in stack_bk:
        return False
    stack = Stack()
    stack.push(src)
    while stack.size:
        popped = stack.pop()
        stack_bk.push(popped)
        for neighbor in graph[popped]:
            if neighbor not in stack_bk:
                stack.push(neighbor)
    return True


def count_components_recursive(graph):
    stack_bk = Stack()
    keys = list(graph.keys())
    def inner(index=0):
        if index == len(keys):
            return 0
        count = 1 if explore_recursive(graph, keys[index], stack_bk) else 0
        return count + inner(index + 1)
    return inner()


def explore_recursive(graph, src, stack_bk):
    def inner(src):
        if src in stack_bk:
            return False
        stack_bk.push(src)
        for neighbor in graph[src]:
            inner(neighbor)
        return True
    return inner(src)
